# 11장

모델 저장 및 읽어오기.
모델은 추론에 써먹으려고 하는것.
학습을 시킨 후 학습에 대한 모델이란것은 파라미터를 최적화시키고 아키텍쳐 정보가 들어가있는것.
최적화된 파라미터.(학습이되어서)
모델들은 웹, OndeviceAI에 쓰인다.

학습을 시켰으면 모델들을 저장하는 방법과 읽어오는것이 필요하다.

-드롭아웃
모델을 훈련시키다보면 과적합되는 때가 있다.
특정한 데이터에 대한 과적합 문제를 해결하기 위한것이 드롭아웃

-CNN메커니즘


**53. 모델 저장 및 읽어오기**

모델이 가지는 매개변수를 외부 파일로 읽고 저장하고 다시 읽어오는 기능을 만든다.

Dezero의 매개변수
-Parameter 클래스로 구현
-Parameter의 데이터는 인스턴스 변수 data에 ndarrray 인스턴스로 보관
-ndarray인스턴스를 외부 파일로 저장

In [2]:
import numpy as np

#np.save와 np.load함수들을 사용하여 ndarray인스턴스를 저장하고 읽을 수 있다.
x = np.array([1,2,3])
np.save('test.npy',x) #ndarray인스턴스를 외부 파일로 저장

x = np.load('test.npy') #저장 되어있는 데이터 읽어온다.
print(x)

[1 2 3]


In [3]:
# 여러 개의 ndarray인스턴스를 저장하고 읽는 방법
x1 = np.array([1,2,3])
x2 = np.array([4,5,6])

np.savez('test.npz', x1=x1, x2=x2) #여러개의 ndarray 인스턴스를 저장, np.savez함수로 정하는 파일 확장자는 .npz

arrays = np.load('test.npz')
x1 = arrays['x1'] #원하는 키워드 명시하여 해당 데이터를 꺼내온다.
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [4]:
# 넘파이의 코드를 파이썬 딕셔너리를 사용하여 수정

x1 = np.array([1,2,3])
x2 = np.array([4,5,6])
data = {'x1':x1, 'x2':x2}
np.savez('test.npy', **data) # 데이터를 저장 # data앞에 **별 두개를 붙여주면 딕셔너리가 자동으로 전개되어 전달됨 

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [ ]:
# # Layer 클래스의 매개변수를 평평하게
# ## Layer 클래스의 계층 구조
# # 계층은 Layer 안에 다른 Layer가 들어가는 중첩형태의 구조

# layer = Layer()

# l1 =  Layer() # 레이어 안에 레이어
# l1.p1 = Parameter(np.array(1)) #레이어안에 레이어p1

# layer.l1 = l1
# layer.p2 = Parameter(np.array(2)) #레이어 안에 레이어
# layer.p3 = Parameter(np.array(3)) #레이어 안에 레이어

In [ ]:
# # _flatten_params 메서드
# # 인수로 딕셔너리인 params_dict와 텍스트인 parent_key를 받는다.
# class Layer:
#     def _flatten_params(self, params_dict, parent_key=""):
#         for name in self._params:
#             obj = self.__dict__[name]
#             key = parent_key + '/' + name if parent_key else name

#             if isinstance(obj, Layer): # 레이어에서 레이어를 꺼냈을 때 파라미터가 아니라면 재귀적으로 다시 호출
#                 obj._flatten_params(params_dict, key)
#             else:
#                 params_dict[key] = obj

In [ ]:
# Layer 클래스의 save함수와 load함수
# def save_weights(self, path):
#         self.to_cpu() #메인 메모리에 존재함 보장

#         params_dict = {}
#         self._flatten_params(params_dict) #평탄화 시킨다.
#         array_dict = {key: param.data for key, param in params_dict.items()
#                     if param is not None}
#         try:
#             np.savez_compressed(path, **array_dict)
#         except (Exception, KeyboardInterrupt) as e:
#             if os.path.exists(path):
#                 os.remove(path)
#             raise

# def load_weights(self, path):
#     npz = np.load(path)
#     params_dict = {}
#     self._flatten_params(params_dict) #값을 꺼내올때
#     for key, param in params_dict.items(): #데이터들을 불러온다.
#         param.data = npz[key]

In [7]:
# MNIST학습으로 매개변수 저장과 읽기 기능 시험
import os
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 3
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10)) #
optimizer = optimizers.SGD().setup(model)

# 매개변수 읽기
if os.path.exists('my_mlp.npz'): # 1번부터 10000번까지 훈련을 시킬때 MNIST를 10개로 구분. # 기존에 훈련한게 있으면 그걸 로드시켜서 돌리겠다.
    model.load_weights('my_mlp.npz')

for epoch in range(max_epoch):
    sum_loss = 0

    for x, t in train_loader:
        y = model(x) #순전파 돌리고
        loss = F.softmax_cross_entropy(y, t) #손실함수 y와 t값으로 비교
        model.cleargrads()
        loss.backward() # 오차
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    print('epoch: {}, loss: {:.4f}'.format(
        epoch + 1, sum_loss / len(train_set)))

# 매개변수 저장하기
model.save_weights('my_mlp.npz')

**54. 드롭아웃과 테스트 모드**

훈련 모델이 99%
테스트 데이터로 하니까 정확도가 70%
>> 이 결과는 이 모델은 해당 데이터에만 과대적합이 일어난다. 

#과대적합이 일어나는 요인
- 훈련 데이터가 적음
- 모델의 표현력이 지나치게 높음

#과대적합 해결방법
- 데이터를 더 확보하거나 인위적으로 늘리는데 확장을 이용하면 됨
- 표현이 높은것은 가중치 감소(표현력이 높은것들을)
- , 드롭아웃(노드들을 없앤다), 배치 정규화(배치값이 실질적으로 데이터 플로우 상에서 고르게 분포되어있게 평탄화시킨다) 등


** 드롭 아웃

드롭아웃은 학습할 때만 노드를 없앤다.
테스트 단계, 추론에서 실제 사용될때는 드롭아웃을 적용하지않는다.

- 은닉층(hidden Layer)뉴런을 무작위로 골라 삭제
- 삭제된 뉴런은 신호를 전송하지 않는다.
- 학습 데이터를 흘려보낼 때마다 삭제할 뉴런을 무작위로 선택

In [ ]:
# 드롭아웃 예시
import numpy as np

dropout_ratio = 0.6
x = np.ones(10)

# 학습 시
mask = np.random.rand(10) > dropout_ratio
y = x * mask # mask에서 값이 False인 원소에 대응하는 x의 원소를 0으로 설정.

# 테스트 시
scale = 1 - dropout_ratio #학습 시에 살아남은 뉴런의 비율
y = x * scale

In [ ]:
# 역 드롭아웃
## 스케일 맞추기를 학습할 때 수행


# 학습 시
scale = 1 - dropout_ratio # dropout_ratio를 동적으로 변경할 수 있다. # 다이렉트 드롭아웃은 dropout_ratio를 고정해두고 학습
mask = np.random.rand(10) > dropout_ratio
y = x * mask / scale # mask에서 값이 False인 원소에 대응하는 x의 원소를 0으로 설정.

# 테스트 시
y = x * scale


In [ ]:
# 테스트 모드 추가
## 역전파 비활성 모드 방식을 유용하게 활용할 수 있음
# class Config:
#     enable_backprop = True
#     train = True


# @contextlib.contextmanager
# def using_config(name, value):
#     old_value = getattr(Config, name)
#     setattr(Config, name, value)
#         yield
#         setattr(Config, name, old_value)

# def test_mode():
#     return using_config('train', False)

In [ ]:
# 드롭아웃 코드 분석

# def dropout(x, dropout_ratio=0.5):
#     x = as_variable(x)

#     if dezero.Config.train:
#         xp = cuda.get_array_module(x)
#         mask = xp.random.rand(*x.shape) > dropout_ratio
#         scale = xp.array(1.0 - dropout_ratio).astype(x.dtype)
#         y = x * mask / scale
#         return y
#     else:
#         return x

In [8]:
# 드롭아웃 구현

import numpy as np
from dezero import test_mode
import dezero.functions as F

x = np.ones(5)
print(x)

# 학습 시 
y = F.dropout(x)
print(y)

# 테스트 시
with test_mode():
    y = F.dropout(x)
    print(y)

[1. 1. 1. 1. 1.]
variable([2. 0. 2. 0. 0.])
variable([1. 1. 1. 1. 1.])


**55.CNN 메커니즘**
- 합성곱 신경망은 이미지 인식, 음성 인식, 자연어 처리 등 다양한 분야에서 사용됨
- 이미지 인식용 딥러닝은 대부분 CNN기반

- CNN의 가장 중요한 구성 요소는 합성곱 층
첫 번째 합성곱 층의 뉴런은 합성곱 층 뉴런의 수용장 안에 있는 픽셀에만 연결

두 번째 합성곱 층에 있는 각 뉴런은 첫 번째 층의 작은 사각 영역 안에 위치한 뉴런에 연결

*CNN 구조
- Conv 계층과 Pool계층이 새로 추가되어서
- Linear -> ReLU연결이 Conx -> ReLU -> (Pool)로 대체
- 출력층에서는 Linear -> ReLU 조합이 사용


*합성곱 연산
- 이미지 처리에서 말하는 필터 연산에 해당(필터를 커널이라고도 쓴다)
- 입력 데이터에 필터를 적용
- 형상을 (높이,너비) 순서대로 표기시 입력 형상은 (4,4) 필터는 (3,3) 출력은 (2,2) 

*연산 계산 순서
- 입력 데이터에 대한 필터 윈도를 일정 간격으로 이동하면서 적용

*편향을 포함한 합성곱 연산의 처리 흐름
- 합성곱층에도 편향이 존재
- 편향은 필터링 후에 더해준다.
- 편향은 하나 뿐이기도하고 하나의 똑같은 값이 필터 적용 후 모든 원소에 브로드캐스트되어 더해진다.

*패딩
- 합성곱층의 주요 처리 전에 입력 데이터 주위에 고정값을 채운다.
- 기본의 4*4에 패딩 1을 해준다면 4*4 주위로 패딩1이 둘러진다.

* 패딩을 하는 이유는 출력크기를 조정!
  IF 9*9인 매트릭스가 존재할때 4*4로 합성곱을 한다면 계속 줄다보니까 7*7...5*5...3*3
  3*3과 4*4합성곱을 못해주니까 패딩 1을 해주는 순간 4*4하고 합성곱이 가능해진다.
세로방향 패딩과 가로방향 패딩을 서로 다르게 설정 할 수 있다.

*스트라이드
인풋(28*28) MNIST SET을 입력으로 집어넣으면 INPUT과 OUTPUT을 연결.
총 결과를 10개로 분류를 한다면 세팅을 할 때 첫번째로 패딩을 세팅. 스트라이드를 2로 설정하면 2만큼 이동하여 계산. 
결론 7*7에 스트라이드를 2로 설정할때 출력 크기는 3*3이 된다.



In [9]:
# 출력 크기 계산
## 패딩 크기를 늘리면 출력 데이터의 크기가 커지고 스트라이드를 크게하면 반대로 작아진다.

# 이건 공식..!!
def get_conv_outsize(input_size, kernel_size, stride, pad):
    return (input_size + pad * 2 - kernel_size) // stride + 1


H, W = 4, 4  # Input size
KH, KW = 3, 3  # Kernel size
SH, SW = 1, 1  # Kernel stride
PH, PW = 1, 1  # Padding size

OH = get_conv_outsize(H, KH, SH, PH)
OW = get_conv_outsize(W, KW, SW, PW)
print(OH, OW)

4 4


**56.CNN메커니즘(2)**
- 2차원 데이터에서의 합성곱 연산
수직 및 수평 방향으로 늘어선 합성곱 연산
가로/세로 방향

- 3차원 데이터의 합성곱 연산
이미지 처리를 위해 3차원 데이터를 다뤄야한다.
깊이 방향으로 데이터가 늘어난것을 제외하면 필터가 움직이는 방법도 계산도 같다.
2차원 합성곱 연산은 대부분의 딥러닝 프레임워크에서 Conv2d라는 이름으로 제공

- 합성곱 연산을 블록으로 표현
데이터가 (채널,높이,너비) 순으로 정렬 되었다면 형상은 (C,H,W) 필터는 (C,KH,KW)로 표기
출력은 특징 맵

- 편향을 포함한 합성곱 연산 처리
편향은 채널당 하나의 값만 갖는다.
편향의 형상은 (OC, 1, 1)이 되고 필터 적용 후의 출력은 (OC, OH, OW)
편향은 형상이 다르기 때문에 브로드캐스트된 다음에 더해진다

- 합성곱 연산의 미니배치 처리
미니배치 처리를 위해서는 각 층을 흐르는 데이터를 4차원 텐서로 취급
N개의 데이터로 이뤄진 미니배치 합성곱 연산 수행
4차원 텐서의 샘플 데이터 각각에 대해 독립적으로 똑같은 합성곱 연산을 수행

- 풀링층
풀링은 가로, 세로 공간을 작게 만드는 연산
최대 풀링은 최댓값을 취하는 연산이며 2 x 2는 대상 영역의 크기를 나타낸다.
일반적으로 풀잉 윈도 크기와 스트라이드 크기는 같은 값으로 설정

* 주요 특징
    학습하는 매개변수가 없다.(대상 영역에서 최댓값(or 평균값)처리만 하면 끝)
    
    채널 수가 변하지 않는다.(계산이 독립적으로 이루어진다.)
    
    미세한 위치 변화에 영향을 덜 받는다.
    (1. 입력 데이터의 차이가 크지 않으면 풀링 결과가 크게 달라지지않는다 
    2. 입력 데이터의 미세한 차이에 강건
   ex. 오른쪽으로 1원소만큼 어긋나지만 출력은 달라지지않는다.)